# Data of Maximum Temperature in California (NOAA)

Author: Martin Pavez

Creation: January 2025

This notebook shows the very beginning steps in heatwave detection from meteorological stations data. 
1. Detection of missing data: quantification and cleaning.
2. Selection of stations. 
3. We generate cleaned data for heatwave detections.

## Libraries import

In [1]:
import numpy as np
import pandas as pd
import os


from datetime import datetime
from calendar import monthrange

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) # Suppress specific RuntimeWarnings

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
os.chdir("../")
# change working directory to project's root path
print(os.getcwd())

c:\Users\marti\Desktop\data\hw_extra


# Parameters


In [4]:
METADATA_PATH_AND_FILENAME = 'data/local_data/NOAA/stations.parquet'
TEMP_DATA_PATH = 'data/local_data/NOAA/original'
CLEANED_DATA_PATH = 'data/local_data/NOAA/cleaned/'

# Utilities


In [5]:
# Function to concatenate CSV files in a folder
def concatenate_csv(folder_path):
    """
    Reads all .csv files in the specified folder, concatenates them (axis=0),
    and indexes the resulting DataFrame by the 'date' column.

    Parameters:
        folder_path (str): Path to the folder containing .csv files.

    Returns:
        pd.DataFrame: Concatenated DataFrame indexed by the 'date' column.
    """
    # List to store DataFrames from each CSV file
    dataframes = []

    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            # Read CSV file, parse 'date' column as datetime
            df = pd.read_csv(file_path, parse_dates=['date'])
            dataframes.append(df)

    # Concatenate all DataFrames along axis 0
    concatenated_df = pd.concat(dataframes, axis=0)

    # Set 'date' column as the index
    concatenated_df.set_index('date', inplace=True)

    ### Found duplicated items, these come from the end and start between files (some have them, some not)
    #duplicates = concatenated_df.index[concatenated_df.index.duplicated(keep=False)]
    #if not duplicates.empty:
    #    print("Duplicate indices found:")
    #    print(duplicates)
    concatenated_df = concatenated_df[~concatenated_df.index.duplicated(keep='first')]


    # Create a complete date range from the earliest to the latest date
    complete_index = pd.date_range(start=concatenated_df.index.min(), end=concatenated_df.index.max(), freq='D')

    # Reindex the DataFrame to the complete date range, filling gaps with NaN
    concatenated_df = concatenated_df.reindex(complete_index)

    # Rename the index to 'date'
    concatenated_df.index.name = 'date'

    # Sort by index for consistency
    concatenated_df.sort_index(inplace=True)

    concatenated_df["value"] = concatenated_df["value"]/10
    concatenated_df.rename(columns={"value":"max_temp"}, inplace=True)

    return concatenated_df

def Month_Days(year):
    return {"year": year, "month_days": {m: monthrange(year, m)[1] for m in range(1, 13)}}

def Tfilter(data, column_label, nperc, year_window_init: int, year_window_end: int):
    start_date = data.index[0]
    end_date = data.index[-1]
    perc_label = 'perc'
    Tadd = 0.0
    year_window_init = year_window_init
    year_window_end = year_window_end
    data_temp = data[column_label]

    data_threshold = pd.DataFrame(
        [],
        columns=[perc_label],
        index = data.index
    )
    for year in range(start_date.year, end_date.year + 1):
        month_days = Month_Days(year)
        for month in month_days["month_days"]:
            for day in range(1, month_days["month_days"][month] + 1):
                try:
                    current_date = datetime(year, month, day)
                except ValueError:
                    if current_date == start_date:
                        current_date = datetime(year, month, day+1)
                    if current_date == end_date:
                        current_date = datetime(year, month, day-1)
                if  current_date >= start_date and current_date <= end_date:
                    f_data_temp = data_temp[
                        (year_window_init <= data_temp.index.year)
                        & (data_temp.index.year <= year_window_end)
                        & (data_temp.index.day == day)
                        & (data_temp.index.month == month)
                    ]
                    try:
                        data_threshold.loc[datetime(year, month, day), perc_label] = f_data_temp.quantile(
                            nperc*0.01, interpolation="midpoint"
                        ).values[0]
                    except AttributeError:
                        data_threshold.loc[datetime(year, month, day), perc_label] = f_data_temp.quantile(
                            nperc*0.01, interpolation="midpoint"
                        )
                    except ValueError:
                        data_threshold.loc[datetime(year, month, day-1), perc_label] = f_data_temp.quantile(
                            nperc*0.01, interpolation="midpoint"
                        ).values[0] + Tadd

        smoothed_series = data_threshold.rolling(window=31, center=True).mean()
    return smoothed_series + Tadd
    #return data_threshold + Tadd

def to_format(data, max_temp_lim = 50, add_filter_year = None, filter_by_hist = False, filter = True, dropnans = True):

    #data["Date"] = pd.to_datetime(data["Date"],format="%Y-%m-%d")

    if dropnans:
        max_temp = data.set_index("date").dropna(subset=["max_temp"])[["max_temp"]]

    #max_temp = max_temp.rename(columns={"DayAirTmpMax":"max_temp"})
    #min_temp = min_temp.rename(columns={"DayAirTmpMin":"min_temp"})
    #mean_temp = mean_temp.rename(columns={"DayAirTmpAvg":"mean_temp"})
    max_temp = data[["max_temp"]]

    if filter:

        if add_filter_year is None:
            max_temp = max_temp.drop(max_temp[np.abs(max_temp["max_temp"])>max_temp_lim].index)
        else:
        #if add_filter_year is not None:
            max_temp.loc[(max_temp.index.year < add_filter_year) & (np.abs(max_temp['max_temp']) > max_temp_lim), 'max_temp'] = np.nan
            max_temp = max_temp.dropna(subset=["max_temp"])[["max_temp"]]


        if filter_by_hist:#add_filter_year is not None:
            perc_max = Tfilter(max_temp, 'max_temp', 99.9, add_filter_year, 2023)
            perc_max = perc_max.reindex(max_temp.index)


            max_temp.loc[(max_temp.index.year < add_filter_year) & (max_temp['max_temp'] > perc_max['perc'] + 10), 'max_temp'] = np.nan

        if dropnans:
            max_temp = max_temp.dropna(subset=["max_temp"])[["max_temp"]]
    
    else:
        return data

    concatenated_df = pd.concat([max_temp], axis=1)

    return concatenated_df

# Reading data

In [6]:
# get all cimis stations information
stations = pd.read_parquet(METADATA_PATH_AND_FILENAME)
stations.head(20)

,elevation,mindate,maxdate,latitude,name,datacoverage,elevationUnit,longitude
id,,,,,,,,
GHCND:USC00040136,516.6,1952-10-01,2024-06-30,32.83580,"ALPINE, CA US",0.9499,METERS,-116.77740
GHCND:USC00040161,1332.9,1905-04-01,2023-03-21,41.49021,"ALTURAS, CA US",0.8324,METERS,-120.54376
GHCND:USC00040192,70.7,1989-08-01,2024-06-30,33.86470,"ANAHEIM, CA US",0.9749,METERS,-117.84250
GHCND:USC00040212,522.7,1940-01-01,2024-07-31,38.57300,"ANGWIN PACIFIC UNION COLLEGE, CA US",0.9201,METERS,-122.44050
GHCND:USC00040332,137.2,2008-08-01,2024-08-08,35.21110,"ARVIN, CA US",0.6777,METERS,-118.83360
GHCND:USC00040343,520.6,1927-01-01,2024-05-31,36.49140,"ASH MOUNTAIN, CA US",0.9478,METERS,-118.82530
GHCND:USC00040383,393.8,1905-01-01,2024-06-28,38.90720,"AUBURN, CA US",0.9179,METERS,-121.08380
GHCND:USC00040444,143.3,1999-01-01,2024-08-28,35.41860,"BAKERSFIELD 5 NW, CA US",0.9423,METERS,-119.05080
GHCND:USC00040449,528.8,1950-02-01,2023-12-31,36.90920,"BALCH POWER HOUSE, CA US",0.9483,METERS,-119.08830


In [19]:
#filter only stations that have data at least from 1971
statlist = list(stations[pd.to_datetime(stations["mindate"])<=  "1971-01-01"].index)
statlist_corrected = [stat.replace(":","_") for stat in statlist]
statlist_corrected

['GHCND_USC00040136',
 'GHCND_USC00040161',
 'GHCND_USC00040212',
 'GHCND_USC00040343',
 'GHCND_USC00040383',
 'GHCND_USC00040449',
 'GHCND_USC00040693',
 'GHCND_USC00040741',
 'GHCND_USC00040790',
 'GHCND_USC00040798',
 'GHCND_USC00040931',
 'GHCND_USC00040943',
 'GHCND_USC00040983',
 'GHCND_USC00041018',
 'GHCND_USC00041072',
 'GHCND_USC00041194',
 'GHCND_USC00041244',
 'GHCND_USC00041253',
 'GHCND_USC00041277']

In [8]:
stations_data_no_filter = {}
stations_data_filter_nans = {}
stations_data_filter1 = {}
dropnans = False

for stat in statlist_corrected:
    print(stat)
    station_data_to_read = concatenate_csv(f"{TEMP_DATA_PATH}/{stat}")[["max_temp"]]
    stations_data_no_filter[stat] = to_format(station_data_to_read, max_temp_lim=50, add_filter_year=None, filter_by_hist = False, filter = False, dropnans=dropnans)
    stations_data_filter1[stat] = to_format(station_data_to_read, max_temp_lim=50, add_filter_year=None, filter_by_hist = False, filter = True, dropnans=dropnans)
    stations_data_filter_nans[stat] = to_format(station_data_to_read, max_temp_lim=50, add_filter_year=None, filter_by_hist = False, filter = False, dropnans=dropnans)

    stations_data_filter1[stat].to_parquet(CLEANED_DATA_PATH + f'Stat_{stat}.parquet')




GHCND_USC00040136
GHCND_USC00040161
GHCND_USC00040212
GHCND_USC00040343
GHCND_USC00040383
GHCND_USC00040449
GHCND_USC00040693
GHCND_USC00040741
GHCND_USC00040790
GHCND_USC00040798
GHCND_USC00040931
GHCND_USC00040943
GHCND_USC00040983
GHCND_USC00041018
GHCND_USC00041072
GHCND_USC00041194
GHCND_USC00041244
GHCND_USC00041253
GHCND_USC00041277


In [37]:
#Percentage of missing and cleaned data

df_nans_and_deleted = pd.DataFrame(index=statlist_corrected)
df_total_days = {}
df_filter1_days = {}
total_days = np.zeros((len(statlist_corrected),))
filter_nan_days = np.zeros((len(statlist_corrected),))
filter1_days = np.zeros((len(statlist_corrected),))

for i, stat in enumerate(statlist_corrected):
    df_total_days = stations_data_no_filter[stat][stations_data_no_filter[stat].index.year>1970]['max_temp']
    #np.isnan(station_data_to_read[stat]['temperature'])
    df_filter_nan_days = stations_data_filter_nans[stat][stations_data_filter_nans[stat].index.year>1970]['max_temp']

    df_filter1_days = stations_data_filter1[stat][stations_data_filter1[stat].index.year>1970]['max_temp']

    #station_data_to_read[stat][np.isnan(station_data_to_read[stat]['temperature'])]
    total_days[i] = len(df_total_days)#[np.isnan(df_total_days)])
    filter_nan_days[i] = len(df_filter_nan_days[np.isnan(df_total_days)])
    filter1_days[i] = len(df_filter1_days[np.isnan(df_filter1_days)])

df_nans_and_deleted['total'] = np.array(total_days)
df_nans_and_deleted['nans'] = np.array(filter_nan_days)/(np.array(total_days)[0])
df_nans_and_deleted['>50'] = -np.array(filter_nan_days)/(np.array(total_days)[0])+np.array(filter1_days)/(np.array(total_days)[0])


print(df_nans_and_deleted)

                     total      nans       >50
GHCND_USC00040136  19266.0  0.119174  0.010589
GHCND_USC00040161  19073.0  0.201183  0.005865
GHCND_USC00040212  18993.0  0.077546  0.014170
GHCND_USC00040343  18353.0  0.061974  0.000311
GHCND_USC00040383  18994.0  0.143984  0.000208
GHCND_USC00040449  18991.0  0.106405  0.008357
GHCND_USC00040693  18994.0  0.097633  0.002855
GHCND_USC00040741  19174.0  0.062753  0.045676
GHCND_USC00040790  10620.0  0.058289  0.000000
GHCND_USC00040798      0.0  0.000000  0.000000
GHCND_USC00040931  19054.0  0.085384  0.019049
GHCND_USC00040943  18968.0  0.127478  0.004100
GHCND_USC00040983  19174.0  0.006644  0.000104
GHCND_USC00041018  19144.0  0.233987  0.004152
GHCND_USC00041072  18922.0  0.151874  0.000104
GHCND_USC00041194  19327.0  0.010641  0.003426
GHCND_USC00041244  18994.0  0.087408  0.019049
GHCND_USC00041253  19052.0  0.016298  0.000104
GHCND_USC00041277  19266.0  0.086733  0.004671


In [38]:
print(r'$\frac{total missing data}{total days} =$' +
      str(np.sum(df_nans_and_deleted['nans']*df_nans_and_deleted['total'])/np.sum(df_nans_and_deleted['total'])))

print(r'$\frac{total missing data}{total days} =$' +
      str(np.sum(df_nans_and_deleted['>50']*df_nans_and_deleted['total'])/np.sum(df_nans_and_deleted['total'])))

$\frac{total missing data}{total days} =$0.09734623533179748
$\frac{total missing data}{total days} =$0.008157624464452805


In [41]:
df_nans_and_deleted.sort_values("nans", ascending=True, inplace=True)
df_nans_and_deleted = df_nans_and_deleted[df_nans_and_deleted["total"] > 18500]
df_nans_and_deleted.iloc[0:10]

,total,nans,>50
GHCND_USC00040983,19174.0,0.006644,0.000104
GHCND_USC00041194,19327.0,0.010641,0.003426
GHCND_USC00041253,19052.0,0.016298,0.000104
GHCND_USC00040741,19174.0,0.062753,0.045676
GHCND_USC00040212,18993.0,0.077546,0.014170
GHCND_USC00040931,19054.0,0.085384,0.019049
GHCND_USC00041277,19266.0,0.086733,0.004671
GHCND_USC00041244,18994.0,0.087408,0.019049
GHCND_USC00040693,18994.0,0.097633,0.002855
GHCND_USC00040449,18991.0,0.106405,0.008357


In [43]:
statlist_10 = list(df_nans_and_deleted.iloc[0:10].index)
statlist_10

['GHCND_USC00040983',
 'GHCND_USC00041194',
 'GHCND_USC00041253',
 'GHCND_USC00040741',
 'GHCND_USC00040212',
 'GHCND_USC00040931',
 'GHCND_USC00041277',
 'GHCND_USC00041244',
 'GHCND_USC00040693',
 'GHCND_USC00040449']

In [44]:
print(str(statlist_10))

['GHCND_USC00040983', 'GHCND_USC00041194', 'GHCND_USC00041253', 'GHCND_USC00040741', 'GHCND_USC00040212', 'GHCND_USC00040931', 'GHCND_USC00041277', 'GHCND_USC00041244', 'GHCND_USC00040693', 'GHCND_USC00040449']
